## Imports and paths

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# import pyogg
import librosa

from tqdm import tqdm # loading bar

from IPython.display import Audio

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


In [2]:
# Check if running on Kaggle
kaggle = ('KAGGLE_KERNEL_RUN_TYPE' in os.environ)
if kaggle:
    # Code specific to Kaggle
    print("Running on Kaggle!")
else:
    print("Not running on Kaggle.")

Not running on Kaggle.


In [3]:
def extract_numbers(filename):
    filename = filename.split('.')[0] # remove extension

    split = filename.split('_')

    if len(split) > 1:
        return split[1]
    elif len(split) == 1:
        return split[0]

In [4]:
if kaggle:
    DATA_DIR = '../input/birdclef-2024/'
    OUTPUT_DIR = '/kaggle/working/'
else: # local work
    DATA_DIR = "../../data/raw/" 
    OUTPUT_DIR = "../../data/processed/"

TRAIN_AUDIO_DIR = os.path.join(DATA_DIR, "train_audio/")

train_csv_path = os.path.join(DATA_DIR, "train_metadata.csv")

# Testing
TEST_AUDIO_DIR = os.path.join(DATA_DIR,"test_soundscapes/")

# Load list of audio files by parsing the test_soundscape folder
test_file_list = sorted(os.listdir(TEST_AUDIO_DIR))
test_file_list = [file for file in test_file_list if file.endswith('.ogg')]  # Filter only ogg files

if len(test_file_list) == 0:   # replace test dir by unlabeled dir for testing
    TEST_AUDIO_DIR = os.path.join(DATA_DIR, "unlabeled_soundscapes/")
    test_file_list = sorted(os.listdir(TEST_AUDIO_DIR))
    test_file_list = [file for file in test_file_list if file.endswith('.ogg')]  # Filter only ogg files
    test_file_list = test_file_list[:5]  # Take only 5 elements to speed up debugging

test_number_list = [extract_numbers(file) for file in test_file_list]

print(f'Directory used for testing: {TEST_AUDIO_DIR}')
print(f'Number of test files: {len(test_file_list)}')

Directory used for testing: ../../data/raw/test_soundscapes/
Number of test files: 1


# Exploratory Data Analysis

In [5]:
train_df = pd.read_csv(train_csv_path)

# Add complete filepath
train_df['filepath'] = train_df.apply(lambda row: os.path.join(TRAIN_AUDIO_DIR, row['filename']), axis=1)

## Train test split

In [6]:
random_state = 43

# Define the number of classes to keep
num_classes_to_keep = 100

# Define the fraction of data to keep for classes with more labels
fraction_to_keep = 0.05

# Calculate the minimum number of instances to keep for classes with fewer labels
min_count = 50

# Calculate weights to balance the classes
class_weights = train_df['primary_label'].value_counts()

# Select the top classes to keep based on their frequencies
top_classes = class_weights.head(num_classes_to_keep).index.tolist()

# Initialize an empty DataFrame to store the sampled subset
train_subset_df = pd.DataFrame()

# Iterate over each class
for label, count in class_weights.items():
    # Check if the class is in the top classes to keep
    if label in top_classes:
        # Check if the class has fewer labels than the minimum count
        if count < min_count:
            # Keep all instances for classes with fewer labels
            subset = train_df[train_df['primary_label'] == label]
        else:
            # Randomly sample a fraction for classes with more labels
            fraction = min(fraction_to_keep, min_count / count)  # Adjust fraction if necessary
            subset = train_df[train_df['primary_label'] == label].sample(frac=fraction, random_state=random_state)
        # Append the subset to the final DataFrame
        train_subset_df = pd.concat([train_subset_df, subset])

# Shuffle the final DataFrame to mix the classes
train_subset_df = train_subset_df.sample(frac=1, random_state=random_state).reset_index(drop=True)


In [7]:
train_subset_df.primary_label.value_counts()

primary_label
blrwar1    25
commoo3    25
grywag     25
lirplo     25
comgre     25
           ..
pursun3     3
grefla1     3
indpit1     3
inbrob1     3
insbab1     3
Name: count, Length: 100, dtype: int64

In [8]:
# Train val split
train_train_df, val_df = train_test_split(train_subset_df, test_size=0.3, stratify = train_subset_df.primary_label, random_state=random_state) 
X_train_files = train_train_df.filepath
X_val_files = val_df.filepath

y_train = train_train_df.primary_label
y_val = val_df.primary_label

In [9]:
y_train.value_counts()

primary_label
bcnher     18
zitcis1    18
graher1    18
comsan     17
woosan     17
           ..
whrmun      2
indpit1     2
grefla1     2
whcbar1     2
rossta2     2
Name: count, Length: 100, dtype: int64

In [10]:
y_val.value_counts()

primary_label
barswa     8
blrwar1    8
bkwsti     8
lirplo     8
eurcoo     8
          ..
shikra1    1
grbeat1    1
compea     1
ashpri1    1
whbwoo2    1
Name: count, Length: 100, dtype: int64

In [11]:
# Label encoding
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Feature extraction

## Method 1

**MFCC (Mel-Frequency Cepstral Coefficients)**

MFCCs are a feature widely used in audio and speech processing. They represent the short-term power spectrum of a sound. The process to calculate MFCCs involves several steps:

- Frame the Signal: The audio signal is divided into short overlapping frames.
- Apply Windowing: Each frame is windowed, typically with a Hamming window, to minimize spectral leakage.
- Calculate the Discrete Fourier Transform (DFT): The Fast Fourier Transform (FFT) is applied to each windowed frame to convert the audio signal from the time domain to the frequency domain.
- Mel Filtering: Mel filtering is applied to the power spectrum to convert the linear frequency scale to the mel scale, which approximates the human auditory system's response to different frequencies.
- Take the Logarithm: The logarithm of the mel-filterbank energies is taken to mimic the human perception of sound intensity.
- Apply Discrete Cosine Transform (DCT): Finally, the Discrete Cosine Transform is applied to the mel-log spectrum to decorrelate the features and obtain the MFCCs.
- MFCCs capture important spectral characteristics of the audio signal and are commonly used as features for tasks like speech recognition, music genre classification, and audio classification.

**Chroma Features**

Chroma features represent the distribution of energy in different pitch classes (i.e., musical notes) within an audio signal. Chroma features are particularly useful for tasks involving music analysis and classification, such as genre classification, chord recognition, and instrument recognition.

The process to compute chroma features involves the following steps:

- Frame the Signal: Similar to MFCCs, the audio signal is divided into short overlapping frames.
- Apply Windowing: Each frame is windowed.
- Calculate the Short-Time Fourier Transform (STFT): The STFT is applied to each windowed frame to obtain the magnitude spectrum.
- Map Frequencies to Chroma: The frequency spectrum is mapped onto the 12 chroma bands corresponding to the 12 pitch classes (C, C#, D, D#, E, F, F#, G, G#, A, A#, B).
- Sum Across Octaves: Chroma features are usually computed by summing the energy within each chroma band across octaves.

Chroma features are robust to changes in pitch and timbre and are commonly used in music information retrieval tasks.

**Mel Spectrogram**

A Mel spectrogram is a spectrogram where the frequencies are converted to the mel scale, similar to the mel filtering step in MFCC computation. It represents the distribution of energy in different frequency bands over time. Mel spectrograms are often used as features for audio classification tasks.

These features capture different aspects of the audio signal's spectral content and are useful for various audio analysis and classification tasks.

In [12]:
def extract_features1(filepath, mfcc=True, chroma=True, mel=True):
    sample_rate = 32000
    audio_data, _ = librosa.load(filepath, sr=sample_rate)
    
    features = []
    
    if mfcc:
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        features.append(np.mean(mfccs, axis=1))
    
    if chroma:
        chroma = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate)
        features.append(np.mean(chroma, axis=1))
    
    if mel:
        mel = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
        features.append(np.mean(mel, axis=1))
    
    return np.concatenate(features)

In [13]:
def extract_features_filepaths1(X_files):
    features = []

    # Wrap the loop with tqdm to add a progress bar
    for filepath in tqdm(X_files, desc='Processing files', total=len(X_files)):

        # Extract features
        audio_features = extract_features1(filepath)

        # Append features and label
        features.append(audio_features)

    X = np.array(features)
    
    return X

In [ ]:
X_train1 = extract_features_filepaths1(X_train_files)

Processing files:  36%|████████████████████████████████▏                                                        | 283/784 [02:52<05:53,  1.42it/s]/home/bfrisque/.pyenv/versions/3.10.6/envs/birdclef2024-lewagon-1601/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Processing files:  65%|██████████████████████████████████████████████████████████                               | 512/784 [04:57<02:44,  1.65it/s]

In [ ]:
X_val1 = extract_features_filepaths1(X_val_files)

In [ ]:
X_train1.shape

In [ ]:
plt.plot(X_train1.T);

## Method 2: improved feature encoding

In [ ]:
def extract_features2(filepath, mfcc=True, chroma=True, mel=True, contrast=True, centroid=True, zero_crossing=True, rms_energy=True, threshold=0.1):
    sample_rate = 32000
    audio_data, _ = librosa.load(filepath, sr=sample_rate)
    
    features = []
    bird_song_segments = []
    
    if rms_energy:
        rms = librosa.feature.rms(y=audio_data)
        rms_mean = np.mean(rms)
        rms_threshold = threshold * rms_mean
        
        # Identify segments where RMS energy exceeds the threshold
        segments = librosa.effects.split(y=audio_data, top_db=rms_threshold)
        
        for segment in segments:
            bird_song_segments.extend(range(segment[0], segment[1] + 1))
    
    if mfcc:
        mfccs = librosa.feature.mfcc(y=audio_data[bird_song_segments], sr=sample_rate, n_mfcc=40)
        features.append(np.mean(mfccs, axis=1))
    
    if chroma:
        chroma = librosa.feature.chroma_stft(y=audio_data[bird_song_segments], sr=sample_rate)
        features.append(np.mean(chroma, axis=1))
    
    if mel:
        mel = librosa.feature.melspectrogram(y=audio_data[bird_song_segments], sr=sample_rate)
        features.append(np.mean(mel, axis=1))
    
    if contrast:
        contrast = librosa.feature.spectral_contrast(y=audio_data[bird_song_segments], sr=sample_rate)
        features.append(np.mean(contrast, axis=1))
    
    if centroid:
        centroid = librosa.feature.spectral_centroid(y=audio_data[bird_song_segments], sr=sample_rate)
        features.append(np.mean(centroid, axis=1))  # Modify this line
    
    if zero_crossing:
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio_data[bird_song_segments])
        features.append(np.mean(zero_crossing_rate, axis=1))  # Modify this line
    
    return np.concatenate(features)


In [ ]:
def extract_features_filepaths2(X_files):
    features = []

    # Wrap the loop with tqdm to add a progress bar
    for filepath in tqdm(X_files, desc='Processing files', total=len(X_files)):

        # Extract features
        audio_features = extract_features2(filepath)

        # Append features and label
        features.append(audio_features)

    X = np.array(features)
    
    return X

In [ ]:
X_train2 = extract_features_filepaths2(X_train_files)

In [ ]:
X_val2 = extract_features_filepaths2(X_val_files)

## Method 3 : Features for CNN

In [ ]:
def extract_features3(filepath, max_length=3000):
    sample_rate = 32000
    audio_data, _ = librosa.load(filepath, sr=sample_rate)
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
    
    # Pad or truncate mfccs to ensure a fixed length
    if mfccs.shape[1] < max_length:
        pad_width = max_length - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    elif mfccs.shape[1] > max_length:
        mfccs = mfccs[:, :max_length]
    
    return mfccs

In [ ]:
def extract_features_filepaths3(X_files):
    features = []

    # Wrap the loop with tqdm to add a progress bar
    for filepath in tqdm(X_files, desc='Processing files', total=len(X_files)):

        # Extract features
        audio_features = extract_features3(filepath)

        # Append features and label
        features.append(audio_features)

    X = np.array(features)
    
    return X

# Model

## Model 1 : Baseline - RandomForest

In [ ]:
# Train a classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train_encoded)

In [ ]:
# Predict
y_pred_encoded = classifier.predict(X_val)

# Evaluate
accuracy = accuracy_score(y_val_encoded, y_pred_encoded)
print(f"Accuracy: {accuracy:.1%}")

## Model 2 : Random forest with feature encoding 2

In [ ]:
# Train a classifier
classifier_bis = RandomForestClassifier(n_estimators=500, random_state=42, verbose=1, n_jobs=-1)
classifier_bis.fit(X_train_bis, y_train_encoded)-

In [ ]:
# Predict
y_pred_encoded_bis = classifier_bis.predict(X_val_bis)

# Evaluate
accuracy_bis = accuracy_score(y_val_encoded, y_pred_encoded_bis)
print(f"Accuracy: {accuracy_bis:.1%}")

## Model 3 : Deep learning with CNN

In [ ]:
X_train3 = extract_features_filepaths3(X_train_files)

In [ ]:
X_val3 = extract_features_filepaths3(X_val_files)

In [ ]:
# Adding number of channels (1)
X_train3 = np.expand_dims(X_train3, -1)
X_val3 = np.expand_dims(X_val3, -1)

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_cat = to_categorical(y_train_encoded)
y_val_cat = to_categorical(y_val_encoded)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define CNN architecture
def create_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = create_model(input_shape=X_train3.shape[1:], num_classes=num_classes_to_keep)
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# Now, you can include the early stopping callback in your model.fit() call
history = model.fit(X_train3, y_train_cat, batch_size=32, epochs=100, validation_data=(X_val3, y_val_cat), callbacks=[early_stopping])

#### Grid search


In [ ]:
# Define the parameter grid for tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a random forest classifier
classifier = RandomForestClassifier(random_state=42)

# Perform grid search cross-validation
grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train_encoded)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Train the classifier with the best parameters
best_classifier = RandomForestClassifier(**best_params, random_state=42)
best_classifier.fit(X_train, y_train_encoded)

In [ ]:
# Define the parameter grid for tuning
param_dist= {
    'n_estimators': stats.randint(250, 400),
    'max_depth': stats.randint(15, 30),
    'min_samples_leaf': [1, 2, 4]
}

# Create a random forest classifier
classifier = RandomForestClassifier(random_state=42)

# Perform grid search cross-validation
random_search = RandomizedSearchCV(estimator=classifier, param_distributions=param_dist, cv=3, n_jobs=-1, verbose=2)
random_search.fit(X_train, y_train_encoded)

# Get the best parameters and the best score
best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

In [ ]:
# Train the classifier with the best parameters
best_classifier = RandomForestClassifier(max_depth=20, min_samples_leaf=2, n_estimators=300, random_state=42)
best_classifier.fit(X_train, y_train_encoded)

In [ ]:
# Predict
y_pred_encoded = best_classifier.predict(X_val)

# # Evaluate
accuracy = accuracy_score(y_val_encoded, y_pred_encoded)
print(f"Accuracy: {accuracy:.1%}")

### XGBoost

In [ ]:
xg_train = xgb.DMatrix(X_train, label=y_train_encoded)
xg_val = xgb.DMatrix(X_val, label=y_val_encoded)

# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 10
param['nthread'] = 4
param['num_class'] = y_train.nunique()


# Specify which dataset and which metric should be used for early stopping.
early_stop = xgb.callback.EarlyStopping(rounds=5,
                                        save_best=True,
                                        data_name='validation')

watchlist = [(xg_train, 'train'), (xg_val, 'validation')]

xgb_classifier = xgb.train(param, xg_train, num_boost_round=500, evals=watchlist, callbacks=[early_stop])


In [ ]:
y_val_pred_encoded = xgb_classifier.predict(dval).astype(np.int32)

# Decode the predicted labels
y_val_pred = label_encoder.inverse_transform(y_val_pred_encoded)

# Evaluate
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Accuracy: {accuracy:.1%}")